In [1]:
pip install opencv-python mediapipe pyautogui


Note: you may need to restart the kernel to use updated packages.


In [ ]:
import cv2
import mediapipe as mp
import pyautogui
import time

# Initialize MediaPipe Hand model and utilities
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands()

# Capture video feed from the webcam
cap = cv2.VideoCapture(0)

# Screen dimensions for controlling the mouse
screen_width, screen_height = pyautogui.size()

# Variables to track clicking state
clicking = False
previous_click_time = 0
click_delay = 0.5  # seconds between clicks to prevent multiple unintended clicks

while True:
    # Read frame from the webcam
    ret, frame = cap.read()
    if not ret:
        break

    # Flip the frame horizontally for a mirror-like effect
    frame = cv2.flip(frame, 1)

    # Convert the frame to RGB (MediaPipe expects RGB)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Process the frame and detect hands
    result = hands.process(rgb_frame)

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            # Get landmark positions of the index finger tip (landmark 8) and thumb tip (landmark 4)
            index_finger_tip = hand_landmarks.landmark[8]
            thumb_tip = hand_landmarks.landmark[4]

            # Convert the normalized hand coordinates to pixel coordinates
            finger_x = int(index_finger_tip.x * screen_width)
            finger_y = int(index_finger_tip.y * screen_height)

            # Move the mouse to the index finger tip's position
            pyautogui.moveTo(finger_x, finger_y)

            # Check for a "click" gesture by measuring the distance between the index finger and thumb
            thumb_x = int(thumb_tip.x * screen_width)
            thumb_y = int(thumb_tip.y * screen_height)

            # Calculate the distance between thumb and index finger
            distance = ((thumb_x - finger_x) ** 2 + (thumb_y - finger_y) ** 2) ** 0.5

            # If the distance is below a certain threshold, perform a click
            if distance < 40:  # Adjust this threshold as needed for better click detection
                current_time = time.time()

                # Ensure sufficient time has passed between clicks to prevent double-clicks
                if not clicking and (current_time - previous_click_time) > click_delay:
                    pyautogui.click()
                    clicking = True
                    previous_click_time = current_time
            else:
                clicking = False

            # Draw the hand landmarks on the frame for visual feedback
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

    # Display the resulting frame
    cv2.imshow('Virtual Mouse', frame)

    # Exit the loop when 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close windows
cap.release()
cv2.destroyAllWindows()


C:\Users\jayap\anaconda3\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '
